# Retention rates for US Universities

This project is being built from the ground up to be customizable and reproducible.

After importing the libraries we need, we're going to load up a table of instructions containing which
of the several thousand attributes we're going to select from the IPEDS database for data mining. This is a lot, so it seems more reasonable to externally store the data list we'll be drawing from.

Many of these, such as website addresses or mission statements, aren't going to be terribly useful. Some others we'll need to exclude as it is too closely related to retention rate, and we risk overfitting or circular logic ("hey, here's how to raise your retention rate--have more of them graduate!")

At the root, each entry in the JSON file denotes a separate table. Included alongside the table name are instructions on whether all the table should be imported as default or not, which attributes are continuous, which are discrete, and  which are strings, and whether multiple records exist for each primary key. This is all derived from the associated documentation that comes with IPEDS. 

We're going to load in each table (obviously checking that it doesn't exist first), and extract the correct tables from it. 

(Should you want to change what we're measuring, you can change the response variable within the JSON file.)

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import scipy
import json
import wget
import sys
import re
import os
from zipfile import ZipFile as zf

Below, we're going to set some variables for how (and where) we process the data.

* *MAXIMUM_NAN* : the ratio of NaNs at which we remove the column entirely.
* *MAXIMUM_COR* : the maximum Pearson correlation that a column can have with another before it's removed.
* *SAMPLE_SIZE* : the fraction of the final dataset to use to build the tree
* *DATA_PATH* : where the data is stored.

In [2]:
MAXIMUM_NAN = 0.80
MAXIMUM_COR = 0.85
SAMPLE_SIZE = 0.90
DATA_PATH = "data"

### Step 1: Reading in and cleaning the IPEDS and ACS data

Next come the functions for reading in the data from files/ZIP archives, corraling them all to one row per unique key, and making sure they're the right data type.

In [3]:
def convert(tdf, columnlist, totype):

# this changes the datatype in each column. where columns have been unstacked, this is supposed
# to change all instances of each unstacked column to that datatype

    for col in columnlist:
        r = re.compile("(^"+col+"*)|(_"+col+")")
        # column name must either be first or have a _ before it, depending on whether we're
        # putting the table names next to it to make it unique
        for c in filter(r.match, list(tdf)): # filter all the column names through this regex
            tdf[c] = tdf[c].astype(totype)

def import_data(filename):
    with open(filename) as file:
        instructions = json.load(file)
        
# does the data directory exist? if no, create it.
    
    if os.path.isdir(DATA_PATH) is False:
        try:  
            os.mkdir(DATA_PATH)
        except OSError:  
            print ("Could not create data folder.")
            raise
            
# start with an empty dataframe to fill, and get the primary key.
# the primary key should be in the first file in the data list
# also figure out if we are adding the tablename to the attributes to make them unique

    ourdata = pd.DataFrame()
    pk = instructions["primarykey"]
    unique_headers = instructions["uniqueheaders"]

# loop through each table. 

    for table in instructions["tables"]:
        
# first, check if the file in the table has been downloaded. if not, download it from
# the path given and throw an error if something is wrong.

        filename = table["name"]+instructions["format"]
        filelocation = DATA_PATH+"/"+filename
        csvfile = DATA_PATH+"/"+table["name"]+".csv"

        if instructions["lowercase"]:
            csvfile = str(csvfile).lower()

        if os.path.exists(filelocation) is False:
            try:
                wget.download(instructions["url"]+filename, filelocation)
            except Exception as e:
                print("Problem downloading and saving", table["name"], ":", e)
                raise

# next, if they are zip files, unzip them

        if instructions["format"] == ".zip":
            if os.path.exists(csvfile) is False:
                print("Unzipping "+table["name"])
                with zf(filelocation,"r") as zip_ref:
                    zip_ref.extractall(DATA_PATH)

# load each CSV file into a temporary data frame, tdf

        if "skiprows" in instructions:
            tdf = pd.read_csv(str(csvfile), encoding = "ISO-8859-1",
                              skiprows = lambda x: x in instructions["skiprows"])
        else:
            tdf = pd.read_csv(str(csvfile), encoding = "ISO-8859-1")

# filter the data according to any values needed

        if "filter" in table:
            for filterinfo in table["filter"]:
                tdf = tdf.loc[tdf[filterinfo[0]] == filterinfo[1]]

# then, depending on the instructions in the JSON file, include all the headers, or include a selection.

        if table["includeall"]:

# include the whole list, excluding the specific tables (might be an empty list)
# and add the primary key to select

            to_include = list(tdf)
            to_exclude = table["exclude"]
            headers = [x for x in to_include if x not in to_exclude]
            headers.append(pk)

# otherwise, stick these three lists together plus the primary key

        else:
            headers = [pk, *table["strings"], *table["discrete"], *table["continuous"]]

        selected_headers = [x for x in tdf.columns if x in headers]

# columns that begin with an X should be removed, at least for now, because they don't describe
# anything other than how the data was collected
# we should also remove any Margin of Error data (_MOE_), it's not needed

        selected_headers = [x for x in selected_headers if not re.search("(?:_MOE_)|(?:^X\s*)", x)]

        # ok, so do we have a primary key? if not, stop right there

        if pk not in tdf.columns:
            raise KeyError("Primary key "+pk+" not found in "+table["name"])

# now we can select the headers we want.

        tdf = tdf[selected_headers]
    
# the code below adds the table name to the headers now, to prevent issues with duplication
# to everythng other than the key

        if unique_headers:
            tdf.rename(columns = lambda x: pk if x == pk else table["name"]+"_"+x, inplace = True)

# next we must check in the JSON instructions if this table contains multiple rows for each
# unique ID. if so, we need to put them all on the same row. to do this, we change them to strings
# then read them into a multiple index, unstack, and then join the column names.

        if "multi" in table:

            if unique_headers:
                multi = [table["name"]+"_"+x for x in table["multi"]]
            else:
                multi = table["multi"]
            tdf[multi] = tdf[multi].astype(str)
            tdf = tdf.set_index([pk, *multi])
            tdf = tdf.unstack(multi)               # need to specify ALL the levels
            tdf.columns = ['_'.join(col) for col in tdf.columns.values]
            tdf = tdf.reset_index(level=pk)

# any '.', '. ', '(X)', '****'', '-' data should be NaN

        tdf = tdf.replace(r"(^\. ?$)|(^\(X\)$)|(^N$)|(^\*{2,}$)|(^-$)", np.nan, regex=True)

# also, apparently some of the ACS data has '-' and '+' signs at the end of some estimates.
# this is not too helpful if we're casting to float, so we'll remove them.

        tdf = tdf.replace(r"(\+$)|(\-$)", "", regex=True)

# and, for some reason, actual commas in numerals. replace them in numbers only, though.

        tdf = tdf.replace(r"(\d+)(,)(\d+)", r"\1\3", regex=True)

# one important thing to do here is to set the right data types--strings or discrete data.
# most of the data in IPEDS and ACS are continuous, but there are a couple of strings and a 
# handful of discrete data.

        if "defaulttype" in table:
            tdfheds = list(tdf) # get list of headers
            hedstoremove = [pk, *table["strings"], *table["discrete"], *table["continuous"]]
            tdfheds = [x for x in tdfheds if x not in hedstoremove]
            if table["defaulttype"] == "discrete":
                convert(tdf, tdfheds, "category")  
            elif table["defaulttype"] == "string":
                convert(tdf, tdfheds, "str")
            else:
                convert(tdf, tdfheds, "float")
                # only way to store NaNs

        convert(tdf, table["strings"], "str")
        convert(tdf, table["discrete"], "category")
        convert(tdf, table["continuous"], "float")
        
        if pk in tdf:
            tdf[pk] = tdf[pk].astype('str') # it is in the index if the data is unstacked

# if it's the first time around the loop, take the first set of data.

        if ourdata.empty:
            ourdata = tdf

# if not, then we need to join on the primary key

        else:
            ourdata = ourdata.merge(tdf,on=pk,how="left")

        print("Imported "+ table["name"]+ ": "+str(len(ourdata.columns))+" columns total, "
              + str(round(float(ourdata.memory_usage().sum() / 1048576), 2)) + "MB")
    
    if "responsevar" in instructions:
        return ourdata, instructions["responsevar"]
    else:
        return ourdata, None

ipeds, responsevar = import_data("ipeds-instructions.json")

Imported HD2016: 27 columns total, 0.4MB
Imported IC2016: 132 columns total, 1.67MB
Imported IC2016_AY: 252 columns total, 8.56MB
Imported ADM2016: 290 columns total, 10.74MB
Imported EFFY2016: 380 columns total, 15.91MB
Imported EF2016A: 650 columns total, 31.4MB
Imported EF2016B: 776 columns total, 38.63MB
Imported EF2016C: 906 columns total, 46.09MB
Imported EF2016D: 917 columns total, 46.72MB
Imported EF2016A_DIST: 962 columns total, 49.3MB
Imported SFA1516: 1285 columns total, 67.83MB
Imported SFAV1516: 1303 columns total, 68.87MB
Imported F1516_F1A: 1413 columns total, 75.13MB
Imported F1516_F2: 1553 columns total, 83.11MB
Imported F1516_F3: 1632 columns total, 87.53MB
Imported EAP2016: 3990 columns total, 222.84MB
Imported SAL2016_IS: 4123 columns total, 230.47MB
Imported SAL2016_NIS: 4151 columns total, 232.08MB
Imported S2016_OC: 4833 columns total, 271.21MB
Imported S2016_SIS: 4910 columns total, 275.63MB
Imported S2016_NH: 4941 columns total, 277.41MB
Imported AL2016: 4971 c

Next, let's import the ACS data. This is 2016 1-year data, put together on a county level by the
American Fact Finder. Unfortunately, this data had to be generated and wasn't available for download.
In order to make this reproducible, I've uploaded the generated files to GitHub.

This data contains info, county-by-county, on employment, healthcare, commuting, income, and even things like how many computers each household has.

In [4]:
acs, x = import_data("acs-instructions.json")

Imported ACS_16_1YR_S0101_with_ann: 109 columns total, 0.69MB
Imported ACS_16_1YR_S0701_with_ann: 389 columns total, 2.47MB
Imported ACS_16_1YR_S0801_with_ann: 560 columns total, 3.56MB
Imported ACS_16_1YR_S1101_with_ann: 660 columns total, 4.19MB
Imported ACS_16_1YR_S1701_with_ann: 843 columns total, 5.35MB
Imported ACS_16_1YR_S1810_with_ann: 1050 columns total, 6.66MB
Imported ACS_16_1YR_S2201_with_ann: 1278 columns total, 8.11MB
Imported ACS_16_1YR_S2301_with_ann: 1418 columns total, 9.0MB
Imported ACS_16_1YR_S2405_with_ann: 1508 columns total, 9.57MB
Imported ACS_16_1YR_S2701_with_ann: 1818 columns total, 11.53MB
Imported ACS_16_1YR_S2702_with_ann: 2028 columns total, 12.86MB
Imported ACS_16_1YR_S2801_with_ann: 2090 columns total, 13.26MB
Imported ACS_16_1YR_S2802_with_ann: 2244 columns total, 14.23MB
Imported ACS_16_1YR_B01003_with_ann: 2246 columns total, 14.25MB
Imported ACS_16_1YR_B05003_with_ann: 2292 columns total, 14.54MB
Imported ACS_16_1YR_B08301_with_ann: 2334 columns tot

We have the two-letter state codes and the county names. We now need to convert them to County Name, State in order to make the join with the American Community Survey data. We'll create a new column and drop the old county name column.

As you can see from the selection below, there's a one-to-many relationship between the counties and the educational institutions. And, of course, many counties won't have institutions in at all...

In [5]:
with open("states_hash.json") as file:
    states = json.load(file)
if "County Name" not in ipeds:
    ipeds["County Name"] = ipeds["COUNTYNM"]+", "+ipeds["STABBR"].map(states)
    ipeds = ipeds.drop(columns="COUNTYNM")
ipeds["County Name"][1000:1025]

1000       Dupage County, Illinois
1001         Cook County, Illinois
1002         Cook County, Illinois
1003         Cook County, Illinois
1004         Cook County, Illinois
1005         Cook County, Illinois
1006         Cook County, Illinois
1007         Cook County, Illinois
1008         Cook County, Illinois
1009         Cook County, Illinois
1010         Cook County, Illinois
1011         Cook County, Illinois
1012         Cook County, Illinois
1013         Cook County, Illinois
1014         Cook County, Illinois
1015         Cook County, Illinois
1016         Cook County, Illinois
1017    Vermilion County, Illinois
1018    Vermilion County, Illinois
1019         Cook County, Illinois
1020       Dupage County, Illinois
1021      Mchenry County, Illinois
1022       Dupage County, Illinois
1023         Cook County, Illinois
1024        Coles County, Illinois
Name: County Name, dtype: object

Let's join them and see what happens.

In [6]:
acs = acs.rename(columns={"GEO.display-label": "County Name"})
ipeds = ipeds.merge(acs,on="County Name",how="left")
ipeds.head()

,UNITID,INSTNM,STABBR,OBEREG,OPEFLAG,ICLEVEL,CONTROL,HLOFFER,UGOFFER,GROFFER,...,ACS_16_1YR_B28011_with_ann_HD01_VD04,ACS_16_1YR_B28011_with_ann_HD02_VD04,ACS_16_1YR_B28011_with_ann_HD01_VD05,ACS_16_1YR_B28011_with_ann_HD02_VD05,ACS_16_1YR_B28011_with_ann_HD01_VD06,ACS_16_1YR_B28011_with_ann_HD02_VD06,ACS_16_1YR_B28011_with_ann_HD01_VD07,ACS_16_1YR_B28011_with_ann_HD02_VD07,ACS_16_1YR_B28011_with_ann_HD01_VD08,ACS_16_1YR_B28011_with_ann_HD02_VD08
0,100654,Alabama A & M University,AL,5,1,1,1,9,1,1,...,102602.0,3229.0,8040.0,1530.0,1146.0,546.0,3293.0,851.0,16880.0,2212.0
1,100663,University of Alabama at Birmingham,AL,5,1,1,1,9,1,1,...,169188.0,4550.0,15666.0,1648.0,6488.0,1442.0,8299.0,1301.0,47189.0,2665.0
2,100690,Amridge University,AL,5,1,1,2,9,1,1,...,59659.0,2858.0,2608.0,614.0,494.0,373.0,1382.0,573.0,16752.0,1601.0
3,100706,University of Alabama in Huntsville,AL,5,1,1,1,9,1,1,...,102602.0,3229.0,8040.0,1530.0,1146.0,546.0,3293.0,851.0,16880.0,2212.0
4,100724,Alabama State University,AL,5,1,1,1,9,1,1,...,59659.0,2858.0,2608.0,614.0,494.0,373.0,1382.0,573.0,16752.0,1601.0


### Step 2. Preparing the data for analysis

This is, clearly, a horrfying amount of data. Let's remove surplus columns before putting them under analysis. First, find out how many NaNs there are for each column and remove those above the threshold. (We should do this before figuring out correlations.)

In [7]:
original_length = len(ipeds.columns)

nanlist = ipeds.isna().sum(axis=0) / len(ipeds)
nans = nanlist.loc[nanlist >= MAXIMUM_NAN]

print (str(round(float(len(nans) / len(ipeds)),4) * 100) + 
       "% of columns removed for missing too much data")

ipeds = ipeds.drop(columns = nans.index)

33.79% of columns removed for missing too much data


Not too much point trying to build a tree around the response variable if it doesn't exist.
Remove the rows where it's missing.

In [10]:
norv = ipeds[responsevar].isna()
ipeds = ipeds.drop(ipeds[norv].index)

Then we need to find out which datasets are too similar to one another, and remove them. 
The most basic way to do this is to do a Pearson correlation between all of the attributes, and
then remove one of each pair, or all-but-one of each collection.

As there are around 5,000 attributes by this point, this will take a little while to process.

In [11]:
# get the pearson correlation between each column. 

cmatrix = ipeds.corr(method="pearson").abs()

# change the matrix to a dataframe, and put the indices in the first two columns

c = cmatrix.unstack().to_frame().reset_index()
c.columns = ["A", "B", "Correlation"]

# remove NaNs
c = c.dropna()

# remove pairs (everything correlates with itself)
c = c.loc[c["A"] != c["B"]]

# and remove everything that is less than the maximum correlation.
c = c.loc[c["Correlation"] >= MAXIMUM_COR]

c.head()

,A,B,Correlation
97,ROOMCAP,ENRLT,0.867458
98,ROOMCAP,ENRLM,0.868164
100,ROOMCAP,ENRLFT,0.874227
101,ROOMCAP,ENRLFTM,0.874670
102,ROOMCAP,ENRLFTW,0.850474


In [12]:
# we now have a list of pairs of variables that correlate with each other.
# now we need to extract which variables to remove from our main dataframe.

# first, let's sort the two columns into alphabetical order so that the first always lies behind
# the second. we do this by applying a lambda across each row, swapping them if they're not in
# order, and broadcasting the result back to the dataframe. 

c = c.apply(lambda row: [row["B"], row["A"], row["Correlation"]] 
                         if row["A"] > row["B"] else [row["A"], row["B"], row["Correlation"]],
                         axis=1, result_type='broadcast')

# we then drop all the duplicates. this should remove probably half or so of the rows.

c = c.drop_duplicates()

# if we then just take the first column, and dedupe that, we'll get a list of columns we can remove.
# if there are more than 2 columns that correlate with one another, one column will always
# only appear in the 'B' list. this will be the one that goes forward to the next stage.

cols_to_remove = c["A"].drop_duplicates()

ipeds = ipeds.drop(columns = cols_to_remove)

print ("Original number of attributes: "+str(original_length))
print ("Reduced number of attributes: "+str(len(ipeds.columns)))
print (str(round(float(len(ipeds.columns) / original_length),4) * 100) + "% of original.")


Original number of attributes: 7524
Reduced number of attributes: 1177
15.64% of original.


That's better.

### Step 3: Build a decision tree

The response variable, PCT_RCF, is continuous -- a percentage from 0-100%. It's most appropriate, therefore, to build a decision tree regressor in order to find out which attributes are most likely to affect the retention rate.

We could decide to classify above or below the mean, but a retention rate is highly contextual based on the college, the environment, and so on -- so there may be different factors in play for higher retention than for lower retention.

Starting with the whole dataset, we'll build training and test sets, set a random seed, and then build a tree out.

But first, we need to fill in the rest of those NaNs. I'm using most common for categorical, and mean for continuous.

In [172]:
regressortree = RandomForestRegressor(random_state=55)

# we have removed a bunch of data and our positional indexing is all messed.
# let's fix that before we sample.

ipeds = ipeds.reset_index().drop(columns = "index")

# remove the NaNs by putting in the column means for floats, and most common for 
# categories

ipeds_f_means = ipeds.select_dtypes(include=["float64", "float"]).apply(np.nanmean, axis=0)
ipeds_c_means = ipeds.select_dtypes(include=["category"]).apply(scipy.stats.mode, axis=0, nan_policy='omit')
ipeds_c_means = ipeds_c_means.iloc[0]

ipeds_f = ipeds.select_dtypes(exclude=["object"])

for col in ipeds_f.select_dtypes(include=["category"]):
    if ipeds_f[col].isnull().sum() > 0:
        ipeds_f[col] = ipeds_f[col].fillna(ipeds_c_means[col][0])

for col in ipeds_f.select_dtypes(include=["float64"]):
    if ipeds_f[col].isnull().sum() > 0:
        ipeds_f[col] = ipeds_f[col].fillna(ipeds_f_means[col])
            
# get the training and test sets, and drop the response variable

ipeds_training = ipeds_f.sample(frac=0.9, random_state=55, axis=0)
test_index = [x for x in list(ipeds_f.index) if x not in list(ipeds_training.index)]
ipeds_test = ipeds_f.iloc[test_index]

/usr/lib/python3/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [173]:
regressortree.fit(ipeds_training.drop(columns = responsevar), ipeds_training[responsevar])

importance = pd.DataFrame(regressortree.feature_importances_)
importance = importance.rename(columns = {0:'Importance'}).sort_values(by="Importance", ascending=False)

/home/quin/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [174]:
importance[0:10]

,Importance
124,0.066557
35,0.042426
16,0.041399
209,0.026000
192,0.021379
203,0.014049
328,0.013736
200,0.013627
153,0.013155
207,0.012902


In [175]:
ipeds_f.columns[importance.index[0:6]]

Index(['HRCHG3', 'LEVEL3', 'C15UGPRF', 'ANYAIDP', 'EFDEEX5 _3', 'UPGRNTP'], dtype='object')

The most important features that contribute to the retention rate, according to this one run of a random forest regression, are as follows:

* **HRCHG3** - Out-of-state per credit hour charge for part-time undergraduates
* **LEVEL3** - Whether the institution offers an associate's degree
* **C15UGPRF** - The Carnegie Undergraduate Profile Classification
* **ANYAIDP** - Percent of full-time first-time undergraduates awarded any financial aid
* **EFDEEX5_3** - Students enrolled exclusively in distance education courses and location of student unknown/not reported; specifically those who are seeking degrees
* **UPGRNTP** - Percent of undergraduate students awarded Pell grants. (Pell grants are for lower-income students.)

Before we test the data, some interesting points:

* Nothing from the American Community Survey has appeared in the top ten, here.
* The feature importances reported aren't terribly high. I don't know whether the low power is because
of the large number of other features, or because none of these are particularly effective.
* I also don't know enough about scikit-learn *yet* to be able to get *p*-values from these features, which would be a bit more useful. I'll do some research.

Secondly, some ways forward.

* This is a single example of a single run of a random forest regression algorithm. In order to make sure this wasn't a fluke, we'll need to run it quite a few times... and perhaps try a couple of other
appropriate algorithms, too. However, the main work of picking, loading and cleaning the data is done.
* As per the literature review, slicing these into college types (private, public / 2-year, 4-year) yielded some more useful results. This is borne out by some of the features that appeared as the most important -- degree type, student type. Can we filter down and get some specific recommendations?
* Some colleges are at 0% first-year retention, and some others are at 100%. Why? Will removing these make the model more reliable, or not?
* Should we ditch using the ACS entirely? I noticed it didn't contain all of the counties. Should we use 5-year data instead?
* I would like those red messages to go away.

Finally, let's test if this model is sound.

In [176]:
ipeds_test["Prediction"] = regressortree.predict(ipeds_test.drop(columns = responsevar))
ipeds_test["squareerror"] = (ipeds_test[responsevar] - ipeds_test["Prediction"])**2
print("Root mean squared error: "+str(round(float(np.sqrt(ipeds_test["squareerror"].mean())), 4)))

Root mean squared error: 16.1444


/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Retention rates are measured in percentages, and a RMSE of +/- 8 percentage points could be worse, I suppose. Let's see if we can get this more accurate with further testing and model refinement. And fewer red warning messages.

~~ Quin Parker

Note: Help was provided by *stackoverflow.com* and *datasciencerosettastone.com*